## Exploratory Data Analysis of Orange County, California Dog License data set and Census Housing.

The Orange County, California dog license data was provided as part of a data request I made. (See 22-3152.pdf) It has about 7700 rows of data on dogs mostly from May and June 2022. A full 12 months would give a more accurate picture of the dogs that are registered in each zip code, but two months seems sufficient to compute the correlations with housing data from the Census.

The data was provided as a text-based pdf file. Luckily, the Camelot package for python can extract the data.
https://camelot-py.readthedocs.io/en/master/

Census data can be found at:

https://data.census.gov/cedsci/table?t=Housing%3AHousing%20Units%3AOccupancy%20Characteristics%3AOwner%2FRenter%20%28Householder%29%20Characteristics%3AOwner%2FRenter%20%28Tenure%29&g=0400000US06_0500000US06059,06059%248600000&y=2020&tid=ACSDP5Y2020.DP04

The downloaded CSV file is: ACSDP5Y2020.DP04_data_with_overlays_2022-07-06T151600.csv

## Conclusion

Similar to what we saw with the Seattle data (https://github.com/MrLRTripp/SeattleHousingwithPets), dog ownership is highly correlated with a zip code's owner occupancy percentage as well as to the zip code's percentage of 4 and 5 bedroom homes. Not surprisingly, owner occupancy percentage is highly correlated with percentage of 4 and 5 bedroom homes.

Unlike the Seattle data, we do see a moderate correlation with property value and rent. Again, because these are moderately correlated with 4 and 5 bedroom homes.

Strong *negative* correlation with areas that have no bedrooms or 1 or 2 bedrooms.

**Bottom line:** households with bigger homes with higher value are most likely to own a dog.

In [6]:
# Requests allows us to make HTTP requests which we will use to get data from an API
import requests
import pandas as pd
import numpy as np

import os

import camelot

import plotly.express as px
import plotly.graph_objects as go
# import make_subplots function from plotly.subplots
# to make grid of plots
from plotly.subplots import make_subplots

In [ ]:
# Use Camelot to extract the data from pdf.
# Since the values in the pdf are separated by whitespace, specify flavor='stream' otherwise it doesn't parse it.
# After experimenting with one page (defaults to 1), provide the pages parameter to get all of them.
# Exclude page 11,31,74
path = 'E:/UserLo/source/repos/learning/OC Dogs/'
file = '22-3152.pdf'


In [338]:
# Use Camelot to extract the data from pdf.
# Since the values in the pdf are separated by whitespace, specify flavor='stream' otherwise it doesn't parse it.
# Camelot parses better if column breaks are specified.
# There are four pages that throw an IndexError when columns are specified: 11,31,74,95
# They all seem to parse when columns are not provided, however there is one row on page 31 that is a problem.
# We'll just drop that row after the dataframe is created

# Each element of the tables list has to be converted to a DataFrame and concatenated to form one large table
license_data = pd.DataFrame(columns=range(6))

for i in range(1,113):
    try:
        tables = camelot.read_pdf(path+file, flavor='stream', pages=str(i), columns=['176,243,367,469,511'])
        license_data = pd.concat([license_data,tables[0].df])
    except IndexError:
        print(f'Problem on page {i}')
        tables = camelot.read_pdf(path+file, flavor='stream', pages=str(i))
        license_data = pd.concat([license_data,tables[0].df])



Problem on page 11
Problem on page 31
Problem on page 74
Problem on page 95


In [301]:
Prob_tables = camelot.read_pdf(path+file, flavor='stream', pages='31')
prob_df = Prob_tables[0].df
prob_df = prob_df.rename(columns={0:'dttmlic', 1:'Species',2:'Primary Breed',3:'Secondary Breed',4:'ZIP Code',5:'Senior'})
prob_df.groupby('Species').count()
prob_df[prob_df['Species'] != 'DOG']

,dttmlic,Species,Primary Breed,Secondary Breed,ZIP Code,Senior
68,6/29/2022 12:00:00 AM W22-092774 DOG,,WEST HIGHLAND,,92675,


In [339]:
license_data = license_data.rename(columns={0:'dttmlic', 1:'Species',2:'Primary Breed',3:'Secondary Breed',4:'ZIP Code',5:'Senior'})
# Drop row 0
license_data = license_data.drop(index=0)
license_data = license_data.reset_index()
license_data.groupby('Species').count()

,index,dttmlic,Primary Breed,Secondary Breed,ZIP Code,Senior
Species,,,,,,
,200,200,200,200,200,200
CAT,60,60,60,60,60,60
DOG,7572,7572,7572,7572,7572,7572


In [340]:
# Let's look at the blank species
license_data[(license_data['Species']!= 'DOG') & (license_data['Species']!= 'CAT')].head()

,index,dttmlic,Species,Primary Breed,Secondary Breed,ZIP Code,Senior
412,63,5/3/2022 12:00:00 AM W22-091933 DOG,,MIX,,92648,
500,11,5/5/2022 12:00:00 AM W22-092017 CAT,,DOMESTIC MH,,92870,
692,63,5/9/2022 12:00:00 AM W22-092100 DOG,,WELSH CORGI PEM,,92807,
2167,68,6/29/2022 12:00:00 AM W22-092774 DOG,,WEST HIGHLAND,,92675,
2175,6,6/1/2022 12:00:00 AM W22-092780 DOG,,GERM WH POINT,,92679,


In [315]:
license_data.loc[412,'dttmlic'].split()

['5/3/2022', '12:00:00', 'AM', 'W22-091933', 'DOG']

In [341]:
# Camelot cannot properly parse those because the date time license and species are too close.
# Split the 'dttmlic' and populate the Species
fix_rows = license_data[(license_data['Species']!= 'DOG') & (license_data['Species']!= 'CAT')].index
license_data.loc[fix_rows,'Species'] = (license_data[(license_data['Species']!= 'DOG') & (license_data['Species']!= 'CAT')]['dttmlic']).str.split(expand=True)[4]
license_data.groupby('Species').count()

,index,dttmlic,Primary Breed,Secondary Breed,ZIP Code,Senior
Species,,,,,,
CAT,66,66,66,66,66,66
DOG,7765,7765,7765,7765,7765,7765


### As a side note...

Camelot provides a way to visualize the text/column parsing. This was used to extablish the columns prameter values

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt

In [285]:
%matplotlib
# Don't use inline so you can zoom in on the plot in another window.
fig = camelot.plot(Prob_tables[0], kind='text')
fig.show()

Using matplotlib backend: QtAgg


In [342]:
# I only want Dog data
drop_other_species = license_data[license_data['Species']!= 'DOG'].index
license_data = license_data.drop(drop_other_species,axis=0)
license_data.groupby('Species').count()

,index,dttmlic,Primary Breed,Secondary Breed,ZIP Code,Senior
Species,,,,,,
DOG,7765,7765,7765,7765,7765,7765


In [345]:
# Drop duplicate index
license_data.drop(columns=['index'],inplace=True)

In [346]:
license_data = pd.concat([license_data,license_data.iloc[:,0].str.split(expand=True)],axis = 1)
license_data = license_data.rename(columns={0:'Date',3:'License Num'})
license_data.drop(columns=[1,2],inplace=True)
license_data.head()

,dttmlic,Species,Primary Breed,Secondary Breed,ZIP Code,Senior,Date,License Num,4
0,5/9/2022 12:00:00 AM L22-599299,DOG,PIT BULL,,92804,Senior,5/9/2022,L22-599299,None
1,5/18/2022 12:00:00 AM L22-468630,DOG,POMERANIAN,,92805,Senior,5/18/2022,L22-468630,None
2,6/29/2022 12:00:00 AM L22-700236,DOG,POODLE MIN,,92886,Senior,6/29/2022,L22-700236,None
3,5/19/2022 12:00:00 AM L22-564964,DOG,WELSH CORGI CAR,MIX,92808,,5/19/2022,L22-564964,None
4,4/7/2022 12:00:00 AM L22-589065,DOG,MIX,,90720,Senior,4/7/2022,L22-589065,None


In [351]:
license_data.drop(columns=[4],inplace=True)

In [352]:
# Drop any rows with missing zip_code
license_data.dropna(axis=0,subset=['ZIP Code'],inplace=True)


In [399]:
license_data['Date']  = license_data['Date'].astype('datetime64')

In [416]:
date_counts = license_data['Date'].value_counts()

In [418]:
sorted_idx = date_counts.index.sort_values(ascending=False)

In [420]:
date_counts = date_counts.reindex(sorted_idx)
date_counts

2022-06-30      6
2022-06-29    134
2022-06-28     81
2022-06-27    136
2022-06-26     50
             ... 
2022-01-16      1
2022-01-14      1
2022-01-12      2
2022-01-11      1
2022-01-03      1
Name: Date, Length: 134, dtype: int64

In [422]:
date_counts.plot(kind='bar')

<AxesSubplot:>

### Use Census DP04 Housing Survey

Use housing stats from the survey to compute:

Percentage of households with dogs in ZCTAs. (Not exactly a percentage of **households** since one household can have multiple dogs, but the public data is not granular enough to determine it perfectly.)

Then answer if percentages are correlated with tenure (rental, owner-occupied), medium home value, median rent, size of home (based on number of bedrooms)

In [359]:
DP04_path = 'E:/UserLo/source/repos/learning/OC Dogs/'
DP04file = 'ACSDP5Y2020.DP04_data_with_overlays_2022-07-06T151600.csv'

DP04_df = pd.read_csv(DP04_path+DP04file)

In [54]:
DP04_df.head()

,DP04_0001E,DP04_0001M,DP04_0001PE,DP04_0001PM,DP04_0002E,DP04_0002M,DP04_0002PE,DP04_0002PM,DP04_0003E,DP04_0003M,...,DP04_0142E,DP04_0142M,DP04_0142PE,DP04_0142PM,DP04_0143E,DP04_0143M,DP04_0143PE,DP04_0143PM,GEO_ID,NAME
0,Estimate!!HOUSING OCCUPANCY!!Total housing units,Margin of Error!!HOUSING OCCUPANCY!!Total hous...,Percent!!HOUSING OCCUPANCY!!Total housing units,Percent Margin of Error!!HOUSING OCCUPANCY!!To...,Estimate!!HOUSING OCCUPANCY!!Total housing uni...,Margin of Error!!HOUSING OCCUPANCY!!Total hous...,Percent!!HOUSING OCCUPANCY!!Total housing unit...,Percent Margin of Error!!HOUSING OCCUPANCY!!To...,Estimate!!HOUSING OCCUPANCY!!Total housing uni...,Margin of Error!!HOUSING OCCUPANCY!!Total hous...,...,Estimate!!GROSS RENT AS A PERCENTAGE OF HOUSEH...,Margin of Error!!GROSS RENT AS A PERCENTAGE OF...,Percent!!GROSS RENT AS A PERCENTAGE OF HOUSEHO...,Percent Margin of Error!!GROSS RENT AS A PERCE...,Estimate!!GROSS RENT AS A PERCENTAGE OF HOUSEH...,Margin of Error!!GROSS RENT AS A PERCENTAGE OF...,Percent!!GROSS RENT AS A PERCENTAGE OF HOUSEHO...,Percent Margin of Error!!GROSS RENT AS A PERCE...,id,Geographic Area Name
1,13315,549,13315,(X),12843,554,96.5,1.3,472,179,...,1553,331,47.4,7.1,228,89,(X),(X),8600000US90620,ZCTA5 90620
2,10709,488,10709,(X),10224,522,95.5,1.8,485,194,...,2712,377,43.7,5.4,209,111,(X),(X),8600000US90621,ZCTA5 90621
3,5057,224,5057,(X),4925,201,97.4,1.7,132,91,...,995,253,59.8,10.1,64,43,(X),(X),8600000US90623,ZCTA5 90623
4,16557,372,16557,(X),15933,410,96.2,1.0,624,163,...,1940,306,43.3,5.8,442,266,(X),(X),8600000US90630,ZCTA5 90630


In [360]:
# Get just the columns of interest
DP04_select_df= DP04_df.loc[:,['DP04_0001PE',
'DP04_0002E',
'DP04_0003E',
'DP04_0039PE',
'DP04_0040PE',
'DP04_0041PE',
'DP04_0042PE',
'DP04_0043PE',
'DP04_0044PE',
'DP04_0046PE',
'DP04_0047PE',
'DP04_0089E',
'DP04_0134E',
'GEO_ID','NAME']]

DP04_select_df.head()


,DP04_0001PE,DP04_0002E,DP04_0003E,DP04_0039PE,DP04_0040PE,DP04_0041PE,DP04_0042PE,DP04_0043PE,DP04_0044PE,DP04_0046PE,DP04_0047PE,DP04_0089E,DP04_0134E,GEO_ID,NAME
0,Percent!!HOUSING OCCUPANCY!!Total housing units,Estimate!!HOUSING OCCUPANCY!!Total housing uni...,Estimate!!HOUSING OCCUPANCY!!Total housing uni...,Percent!!BEDROOMS!!Total housing units!!No bed...,Percent!!BEDROOMS!!Total housing units!!1 bedroom,Percent!!BEDROOMS!!Total housing units!!2 bedr...,Percent!!BEDROOMS!!Total housing units!!3 bedr...,Percent!!BEDROOMS!!Total housing units!!4 bedr...,Percent!!BEDROOMS!!Total housing units!!5 or m...,Percent!!HOUSING TENURE!!Occupied housing unit...,Percent!!HOUSING TENURE!!Occupied housing unit...,Estimate!!VALUE!!Owner-occupied units!!Median ...,Estimate!!GROSS RENT!!Occupied units paying re...,id,Geographic Area Name
1,13315,12843,472,1.6,3.2,16.6,49.0,25.9,3.7,72.7,27.3,607500,1892,8600000US90620,ZCTA5 90620
2,10709,10224,485,3.9,13.8,38.4,24.0,15.8,4.2,37.2,62.8,617100,1652,8600000US90621,ZCTA5 90621
3,5057,4925,132,1.5,8.2,18.7,26.3,36.2,9.1,64.9,35.1,688000,1854,8600000US90623,ZCTA5 90623
4,16557,15933,624,0.6,6.2,19.2,36.9,29.8,7.3,69.1,30.9,664700,1969,8600000US90630,ZCTA5 90630


In [361]:
# Drop metadata row 0
DP04_select_df.drop(index=0,inplace=True)
DP04_select_df.head()

,DP04_0001PE,DP04_0002E,DP04_0003E,DP04_0039PE,DP04_0040PE,DP04_0041PE,DP04_0042PE,DP04_0043PE,DP04_0044PE,DP04_0046PE,DP04_0047PE,DP04_0089E,DP04_0134E,GEO_ID,NAME
1,13315,12843,472,1.6,3.2,16.6,49.0,25.9,3.7,72.7,27.3,607500,1892,8600000US90620,ZCTA5 90620
2,10709,10224,485,3.9,13.8,38.4,24.0,15.8,4.2,37.2,62.8,617100,1652,8600000US90621,ZCTA5 90621
3,5057,4925,132,1.5,8.2,18.7,26.3,36.2,9.1,64.9,35.1,688000,1854,8600000US90623,ZCTA5 90623
4,16557,15933,624,0.6,6.2,19.2,36.9,29.8,7.3,69.1,30.9,664700,1969,8600000US90630,ZCTA5 90630
5,22251,21548,703,2.5,11.2,30.1,33.0,17.7,5.5,60.8,39.2,607300,1649,8600000US90631,ZCTA5 90631


In [362]:
# The built-in str.isnumeric() returns False for floating point, so use a custom function
def is_number(n):
    try:
        float(n)   # Type-casting the string to `float`.
                   # If string is not a valid `float`, 
                   # it'll raise `ValueError` exception
    except ValueError:
        return False
    return True

In [363]:
# Find any non-numeric values in the columns that will be converted to numeric
numcheck = ~DP04_select_df.loc[:,['DP04_0001PE',
'DP04_0002E',
'DP04_0003E',
'DP04_0039PE',
'DP04_0040PE',
'DP04_0041PE',
'DP04_0042PE',
'DP04_0043PE',
'DP04_0044PE',
'DP04_0046PE',
'DP04_0047PE',
'DP04_0089E',
'DP04_0134E']].applymap(is_number).all(axis=1)


In [364]:
DP04_select_df[numcheck]

,DP04_0001PE,DP04_0002E,DP04_0003E,DP04_0039PE,DP04_0040PE,DP04_0041PE,DP04_0042PE,DP04_0043PE,DP04_0044PE,DP04_0046PE,DP04_0047PE,DP04_0089E,DP04_0134E,GEO_ID,NAME
12,262,120,142,5.3,3.8,27.9,40.5,18.7,3.8,67.5,32.5,"2,000,000+",-,8600000US90743,ZCTA5 90743
27,6819,5895,924,3.2,7.3,24.4,34.7,21.0,9.4,66.4,33.6,"2,000,000+",3092,8600000US92625,ZCTA5 92625
41,4152,3689,463,0.0,4.8,16.8,23.4,29.8,25.2,80.2,19.8,"2,000,000+",2660,8600000US92657,ZCTA5 92657
43,2798,1827,971,2.3,7.1,39.2,22.9,18.2,10.3,39.4,60.6,"2,000,000+",2282,8600000US92661,ZCTA5 92661
44,1981,1297,684,2.7,8.1,27.4,40.3,18.6,2.9,59.1,40.9,"2,000,000+",2141,8600000US92662,ZCTA5 92662
85,2046,1972,74,0.0,1.1,0.9,15.8,50.3,31.9,94.5,5.5,1239600,-,8600000US92861,ZCTA5 92861


In [365]:
# Two rent values are missing. Replace them with mean of the values that are provided
mean_rent = DP04_select_df[DP04_select_df['DP04_0134E'].map(is_number)]['DP04_0134E'].astype('float64').mean()
DP04_select_df = DP04_select_df.replace(to_replace={'DP04_0134E':'-'},  value=mean_rent)

In [366]:
# There are a few non-numeric home values for those $2M or more.
# Just replace '2,000,000+' with '2000000' so it can be converted to numeric type
DP04_select_df = DP04_select_df.replace(to_replace={'DP04_0089E':'2,000,000+'},  value='2000000')

In [367]:
# Convert numberical columns to numeric type
DP04_select_df = DP04_select_df.astype({'DP04_0001PE':'float64',
'DP04_0002E':'float64',
'DP04_0003E':'float64',
'DP04_0039PE':'float64',
'DP04_0040PE':'float64',
'DP04_0041PE':'float64',
'DP04_0042PE':'float64',
'DP04_0043PE':'float64',
'DP04_0044PE':'float64',
'DP04_0046PE':'float64',
'DP04_0047PE':'float64',
'DP04_0089E':'float64',
'DP04_0134E':'float64'})

DP04_select_df.dtypes

DP04_0001PE    float64
DP04_0002E     float64
DP04_0003E     float64
DP04_0039PE    float64
DP04_0040PE    float64
DP04_0041PE    float64
DP04_0042PE    float64
DP04_0043PE    float64
DP04_0044PE    float64
DP04_0046PE    float64
DP04_0047PE    float64
DP04_0089E     float64
DP04_0134E     float64
GEO_ID          object
NAME            object
dtype: object

In [368]:
# Get the ZCTA out of NAME
DP04_select_df['ZCTA'] = DP04_select_df['NAME'].str[-5:]

For each row in DP04_select, append count of dogs in the ZCTA.

In [369]:
zip_size = license_data[['ZIP Code','License Num']].groupby(['ZIP Code']).count()

In [370]:
zip_size

,License Num
ZIP Code,
90630,284
90720,109
90742,5
90747,1
92610,111
92627,3
92630,361
92646,461
92647,326


In [371]:
# Join DP04_select_df and zip_size on the zip code
DP04_select_df = DP04_select_df.merge(zip_size,how='inner',left_on='ZCTA',right_index=True)
DP04_select_df = DP04_select_df.rename(columns={'License Num':'Count'})
DP04_select_df.head()

,DP04_0001PE,DP04_0002E,DP04_0003E,DP04_0039PE,DP04_0040PE,DP04_0041PE,DP04_0042PE,DP04_0043PE,DP04_0044PE,DP04_0046PE,DP04_0047PE,DP04_0089E,DP04_0134E,GEO_ID,NAME,ZCTA,Count
4,16557.0,15933.0,624.0,0.6,6.2,19.2,36.9,29.8,7.3,69.1,30.9,664700.0,1969.0,8600000US90630,ZCTA5 90630,90630,284
9,8179.0,8029.0,150.0,0.9,3.0,22.4,41.8,27.6,4.3,66.6,33.4,925800.0,2040.0,8600000US90720,ZCTA5 90720,90720,109
11,530.0,357.0,173.0,6.2,17.0,36.0,25.1,8.7,7.0,38.4,61.6,1135400.0,1914.0,8600000US90742,ZCTA5 90742,90742,5
20,6113.0,5774.0,339.0,1.0,8.8,19.5,36.0,25.2,9.5,70.4,29.6,793300.0,2387.0,8600000US92610,ZCTA5 92610,92610,111
29,22508.0,21706.0,802.0,3.6,17.1,39.9,29.1,8.3,2.0,39.2,60.8,817100.0,1829.0,8600000US92627,ZCTA5 92627,92627,3


In [372]:
DP04_select_df.dtypes

DP04_0001PE    float64
DP04_0002E     float64
DP04_0003E     float64
DP04_0039PE    float64
DP04_0040PE    float64
DP04_0041PE    float64
DP04_0042PE    float64
DP04_0043PE    float64
DP04_0044PE    float64
DP04_0046PE    float64
DP04_0047PE    float64
DP04_0089E     float64
DP04_0134E     float64
GEO_ID          object
NAME            object
ZCTA            object
Count            int64
dtype: object

In [374]:
DP04_select_df['households_with_dog_pct'] = (DP04_select_df['Count']/DP04_select_df['DP04_0002E'])*100
DP04_select_df.head()

,DP04_0001PE,DP04_0002E,DP04_0003E,DP04_0039PE,DP04_0040PE,DP04_0041PE,DP04_0042PE,DP04_0043PE,DP04_0044PE,DP04_0046PE,DP04_0047PE,DP04_0089E,DP04_0134E,GEO_ID,NAME,ZCTA,Count,households_with_dog_pct
4,16557.0,15933.0,624.0,0.6,6.2,19.2,36.9,29.8,7.3,69.1,30.9,664700.0,1969.0,8600000US90630,ZCTA5 90630,90630,284,1.782464
9,8179.0,8029.0,150.0,0.9,3.0,22.4,41.8,27.6,4.3,66.6,33.4,925800.0,2040.0,8600000US90720,ZCTA5 90720,90720,109,1.357579
11,530.0,357.0,173.0,6.2,17.0,36.0,25.1,8.7,7.0,38.4,61.6,1135400.0,1914.0,8600000US90742,ZCTA5 90742,90742,5,1.400560
20,6113.0,5774.0,339.0,1.0,8.8,19.5,36.0,25.2,9.5,70.4,29.6,793300.0,2387.0,8600000US92610,ZCTA5 92610,92610,111,1.922411
29,22508.0,21706.0,802.0,3.6,17.1,39.9,29.1,8.3,2.0,39.2,60.8,817100.0,1829.0,8600000US92627,ZCTA5 92627,92627,3,0.013821


### Let's make some visualizations

Compute percentage of households with dogs using Total Occupied Housing Units DP04_0002E.

| Column ID        | Description           |
| ------------- |:-------------:|
| DP04_0046PE | Percent!!HOUSING TENURE!!Occupied housing units!!Owner-occupied |
| DP04_0047PE | Percent!!HOUSING TENURE!!Occupied housing units!!Renter-occupied |
| DP04_0089E | Estimate!!VALUE!!Owner-occupied units!!Median (dollars) |
| DP04_0134E |	Estimate!!GROSS RENT!!Occupied units paying rent!!Median (dollars) |
| DP04_0039PE |	Percent!!BEDROOMS!!Total housing units!!No bedroom |
| DP04_0040PE |	Percent!!BEDROOMS!!Total housing units!!1 bedroom |
| DP04_0041PE |	Percent!!BEDROOMS!!Total housing units!!2 bedrooms |
| DP04_0042PE |	Percent!!BEDROOMS!!Total housing units!!3 bedrooms |
| DP04_0043PE |	Percent!!BEDROOMS!!Total housing units!!4 bedrooms |
| DP04_0044PE |	Percent!!BEDROOMS!!Total housing units!!5 or more bedrooms |



In [375]:
# Rename the columns of interest to match description
DP04_select_df.rename(columns={'DP04_0046PE':'Pct Owner Occ','DP04_0047PE':'Pct Renter Occ','DP04_0089E':'Owner Occ Median Val',
'DP04_0134E':'Median Rent','DP04_0039PE':'Pct No Bedroom','DP04_0040PE':'Pct 1 Bedroom','DP04_0041PE':'Pct 2 Bedrooms','DP04_0042PE':'Pct 3 Bedrooms',
'DP04_0043PE':'Pct 4 Bedrooms','DP04_0044PE':'Pct 5+ Bedrooms'},inplace=True)

In [376]:
DP04_select_df.head()

,DP04_0001PE,DP04_0002E,DP04_0003E,Pct No Bedroom,Pct 1 Bedroom,Pct 2 Bedrooms,Pct 3 Bedrooms,Pct 4 Bedrooms,Pct 5+ Bedrooms,Pct Owner Occ,Pct Renter Occ,Owner Occ Median Val,Median Rent,GEO_ID,NAME,ZCTA,Count,households_with_dog_pct
4,16557.0,15933.0,624.0,0.6,6.2,19.2,36.9,29.8,7.3,69.1,30.9,664700.0,1969.0,8600000US90630,ZCTA5 90630,90630,284,1.782464
9,8179.0,8029.0,150.0,0.9,3.0,22.4,41.8,27.6,4.3,66.6,33.4,925800.0,2040.0,8600000US90720,ZCTA5 90720,90720,109,1.357579
11,530.0,357.0,173.0,6.2,17.0,36.0,25.1,8.7,7.0,38.4,61.6,1135400.0,1914.0,8600000US90742,ZCTA5 90742,90742,5,1.400560
20,6113.0,5774.0,339.0,1.0,8.8,19.5,36.0,25.2,9.5,70.4,29.6,793300.0,2387.0,8600000US92610,ZCTA5 92610,92610,111,1.922411
29,22508.0,21706.0,802.0,3.6,17.1,39.9,29.1,8.3,2.0,39.2,60.8,817100.0,1829.0,8600000US92627,ZCTA5 92627,92627,3,0.013821


In [377]:
plot_df = DP04_select_df.copy()
plot_df = plot_df.astype({'ZCTA':'string'})

# use specs parameter in make_subplots function
# to create secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])
 
# plot a scatter chart by specifying the x and y values
# Use add_trace function to specify secondary_y axes.
fig.add_trace(
    go.Scatter(x=plot_df['ZCTA'], y=plot_df['households_with_dog_pct'], name='Pct Households with Dog', mode='lines+markers'),
    secondary_y=False)
 
# Use add_trace function and specify secondary_y axes = True.
fig.add_trace(
    go.Scatter(x=plot_df['ZCTA'], y=plot_df['Pct Owner Occ'], name='Owner Occ Pct',mode='lines+markers'),
    secondary_y=True,)


fig.update_xaxes(autotypenumbers='strict')  # Don't convert zip codes to numbers
fig.update_xaxes(title_text='ZIP Code')
fig.update_yaxes(title_text='Pct Households with Dog',secondary_y=False)
fig.update_yaxes(title_text='Pct Owner Occ', secondary_y=True)
fig.update_layout(legend_title='Demographics')
#fig.update_layout(title=title)
fig.show()

### We can see there is some variation based on ZIP Code

Let's look at the demographics to see if there is a correlation with dog ownership percentage. As we see below, the highest positive correlation is with Owner Occupancy with Pct Dog Ownership > 4%. *Less than 4% is too noisy*

In [378]:
def plot_attribute(col_name):

    fig = px.scatter(DP04_select_df,x='households_with_dog_pct', y=[col_name]) 
    #fig.update_traces(mode='lines+markers')

    #fig.update_xaxes(autotypenumbers=autotype)
    fig.update_xaxes(title_text='Percent with dog')
    fig.update_yaxes(title_text='Percentage')
    fig.update_layout(legend_title='Demographics')
    #fig.update_layout(title=title)
    fig.show()




In [390]:
plot_attribute('Pct Owner Occ')

In [379]:
plot_attribute('Pct Renter Occ')


In [391]:
plot_attribute('Pct No Bedroom')

In [380]:
plot_attribute('Pct 1 Bedroom')

In [381]:
plot_attribute('Pct 2 Bedrooms')

In [382]:
plot_attribute('Pct 3 Bedrooms')

In [383]:
plot_attribute('Pct 4 Bedrooms')

In [384]:
plot_attribute('Pct 5+ Bedrooms')

In [385]:
def plot_attribute_dollar(col_name):

    fig = px.scatter(DP04_select_df,x='households_with_dog_pct', y=[col_name]) 
    #fig.update_traces(mode='lines+markers')

    #fig.update_xaxes(autotypenumbers=autotype)
    fig.update_xaxes(title_text='Percent with dog')
    fig.update_yaxes(title_text='Amount (Dollars)')
    fig.update_layout(legend_title='Demographics')
    #fig.update_layout(title=title)
    fig.show()




In [392]:
plot_attribute_dollar('Owner Occ Median Val')

In [393]:
plot_attribute_dollar('Median Rent')

In [394]:
# From the visualizations, there doesn't appear to be any correlation when 'households_with_dog_pct' < 1.0]
# Let's just look at the correlations when 'households_with_dog_pct' >= 1.0]
DP04_select_ge1_df = DP04_select_df[DP04_select_df['households_with_dog_pct'] >= 1.0]

In [395]:
DP04_select_ge1_df[['households_with_dog_pct','Pct No Bedroom',
       'Pct 1 Bedroom', 'Pct 2 Bedrooms', 'Pct 3 Bedrooms', 'Pct 4 Bedrooms',
       'Pct 5+ Bedrooms', 'Pct Owner Occ', 'Pct Renter Occ',
       'Owner Occ Median Val', 'Median Rent']].corr()

,households_with_dog_pct,Pct No Bedroom,Pct 1 Bedroom,Pct 2 Bedrooms,Pct 3 Bedrooms,Pct 4 Bedrooms,Pct 5+ Bedrooms,Pct Owner Occ,Pct Renter Occ,Owner Occ Median Val,Median Rent
households_with_dog_pct,1.000000,-0.726170,-0.780737,-0.750665,0.052361,0.772167,0.754760,0.845382,-0.845382,0.540554,0.612641
Pct No Bedroom,-0.726170,1.000000,0.815648,0.671603,-0.278925,-0.744071,-0.626424,-0.853763,0.853763,-0.403941,-0.619127
Pct 1 Bedroom,-0.780737,0.815648,1.000000,0.774296,-0.398553,-0.832930,-0.657272,-0.909979,0.909979,-0.526199,-0.652002
Pct 2 Bedrooms,-0.750665,0.671603,0.774296,1.000000,-0.088299,-0.949280,-0.735039,-0.822140,0.822140,-0.516642,-0.551818
Pct 3 Bedrooms,0.052361,-0.278925,-0.398553,-0.088299,1.000000,0.052522,-0.289484,0.201701,-0.201701,-0.204626,0.162831
Pct 4 Bedrooms,0.772167,-0.744071,-0.832930,-0.949280,0.052522,1.000000,0.691957,0.836644,-0.836644,0.515454,0.592134
Pct 5+ Bedrooms,0.754760,-0.626424,-0.657272,-0.735039,-0.289484,0.691957,1.000000,0.799530,-0.799530,0.711311,0.519151
Pct Owner Occ,0.845382,-0.853763,-0.909979,-0.822140,0.201701,0.836644,0.799530,1.000000,-1.000000,0.525259,0.652166
Pct Renter Occ,-0.845382,0.853763,0.909979,0.822140,-0.201701,-0.836644,-0.799530,-1.000000,1.000000,-0.525259,-0.652166
Owner Occ Median Val,0.540554,-0.403941,-0.526199,-0.516642,-0.204626,0.515454,0.711311,0.525259,-0.525259,1.000000,0.444171
